In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [3]:
data=pd.read_csv('E:/Projects/easy spam detector/spam.csv',encoding='latin-1')

In [4]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
data = data[['v1','v2']]
data=data.rename(columns= {'v1':'label','v2':'text'})

In [6]:
data_lemma=data

In [7]:
def review_messages(msg):
    msg=msg.lower()
    return msg

In [8]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [9]:
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))

In [10]:
def alternate_review_messages(msg):
    msg=msg.lower()
    msg= [ word for word in msg.split() if word not in stopwords]
    
    #using lemmatization:
    msg= " ".join([lemmatizer.lemmatize(word) for word in msg])
    
    return msg

In [11]:
# first func to normalise the text
data['text'] = data['text'].apply(review_messages)

In [12]:
data.head()

,label,text
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."


In [13]:
data_lemma['text']=data['text'].apply(alternate_review_messages)

In [14]:
data_lemma.head()

,label,text
0,ham,"go jurong point, crazy.. available bugis n gre..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor... u c already say...
4,ham,"nah think go usf, life around though"


In [25]:
data.loc[:,'label'] = LabelEncoder.fit_transform(data,data['label'])
#ohe = OneHotEncoder(categorical_features=[0,1])
#data = ohe.fit_transform(data.loc[:,'label']).to_array()

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size = 0.1, random_state = 1)

In [29]:
#training TF-IDF vectorizer:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer =TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)

In [30]:
X_test = vectorizer.transform(X_test)

In [31]:
from sklearn import svm
svm = svm.SVC(C=1000)
svm.fit(X_train, y_train)

C:\Users\dell\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [32]:
y_pred = svm.predict(X_test)

In [33]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_pred))

[[490   0]
 [ 12  56]]


In [34]:
from sklearn.metrics import precision_score,recall_score

In [35]:
from sklearn.metrics import accuracy_score

In [36]:
accuracy_score(y_test,y_pred)

0.978494623655914

In [37]:
precision_score(y_test,y_pred)

1.0

In [38]:
recall_score(y_test,y_pred)

0.8235294117647058

In [39]:
def pred(msg):
    msg=vectorizer.transform([msg])
    prediction = svm.predict(msg)
    if prediction[0]==0:
        return 'ham'
    else:
        return 'spam'

In [40]:
pred("You HAVE won 20 million CASH PRIZE")

'spam'

In [41]:
pred("How are You ?")

'ham'